Importing libraries

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

Import the dataset

In [4]:
dataset= pd.read_csv('train.csv')
X= dataset.iloc[:,:-1].values
y= dataset.iloc[:, -1].values

In [5]:
print(X.shape[1])
print(X.shape[0]*0.25)
print(int(X.shape[0]*0.25))

81
5315.75
5315


Splitting the data into training and test sets

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size=0.25, random_state=0)

Applying Standard Scaling

In [7]:
from sklearn.preprocessing import StandardScaler
sc= StandardScaler()
X_train= sc.fit_transform(X_train)
X_test= sc.transform(X_test)

Applying PCA

In [8]:
from sklearn.decomposition import PCA
pca= PCA(n_components='mle')
X_train= pca.fit_transform(X_train)
X_test= pca.transform(X_test)

Training the SVR Model 

In [9]:
from sklearn.svm import SVR
svr= SVR(kernel='rbf')
svr.fit(X_train, y_train)

SVR()

Evaluation of SVR model

In [10]:
from sklearn.metrics import r2_score
y_pred= svr.predict(X_test)
score= r2_score(y_test, y_pred)
print("The r2 SCORE is " + str(score))

The r2 SCORE is 0.76605348786781


Using Grid search to find the optimal SVR model 

In [11]:
from sklearn.metrics import make_scorer
def adjustedr2(y_test, y_pred):
  r2= r2_score(y_test, y_pred)
  p= X.shape[1]
  n= int(X.shape[0]*0.25)
  adjr2= 1-(1-r2)*(n-1)/(n-p-1)
  return adjr2

adjustedr2score= make_scorer(adjustedr2, greater_is_better= True)

In [12]:
from sklearn.model_selection import GridSearchCV
parameters= [{'C': [0.25, 0.5, 0.75, 1], "kernel": ["linear"]}, 
             {'C': [0.25, 0.5, 0.75, 1], "kernel": ["rbf"], "gamma": [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]},
             {'C': [0.25, 0.5, 0.75, 1], "kernel": ["poly"], "degree": [3,4,5], 
              "coef0": [0.0, 0.2, 0.4, 0.6, 0.8, 1.0]}]
gridsearch= GridSearchCV(estimator= svr, param_grid= parameters, scoring= adjustedr2score,
                         cv= 5, n_jobs= -1)
gridsearch.fit(X_train, y_train)
best_r2score= gridsearch.best_score_
best_param= gridsearch.best_params_ 
print("Best adjusted r2 score: {:.4f}".format(best_r2score))
print("Best parameters: ", best_param)              

Best adjusted r2 score: 0.8198
Best parameters:  {'C': 1, 'coef0': 1.0, 'degree': 5, 'kernel': 'poly'}


In [13]:
# from sklearn.model_selection import GridSearchCV
# parameters= [{'C': [0.25, 0.5, 0.75, 1], "kernel": ["linear"], 'epsilon': [0.1, 0.2, 0.3, 0.4]}, 
#              {'C': [0.25, 0.5, 0.75, 1], "kernel": ["rbf"], "gamma": [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9], 
#               'epsilon': [0.1, 0.2, 0.3, 0.4]},
#              {'C': [0.25, 0.5, 0.75, 1], "kernel": ["poly"], "degree": [3,4,5,6,7,8,9,10], 
#               "coef0": [0.0, 0.2, 0.4, 0.6, 0.8, 1.0], 'epsilon': [0.1, 0.2, 0.3, 0.4]}]
# gridsearch= GridSearchCV(estimator= svr, param_grid= parameters, scoring= make_scorer(adjustedr2, greater_is_better= True),
#                          cv= 5, n_jobs= -1)
# gridsearch.fit(X_train, y_train)
# best_r2score= gridsearch.best_score_
# best_param= gridsearch.best_params_ 
# print("Best r2 score: {:.4f}".format(best_r2score))
# print("Best parameters: ", best_param)              

In [14]:
# comp= [2,3]

# for i in comp:
#   pca= PCA(n_components=comp[i-1])
#   X_train= pca.fit_transform(X_train)
#   X_test= pca.transform(X_test)
#   svr.fit(X_train, y_train)
#   y_pred= svr.predict(X_test)
#   score= r2_score(y_test, y_pred)
#   print("For "+ str(comp[i]) +" components")
#   print("The r2 SCORE is " + str(score))